# **Imports and Libraries**

In [6]:
!pip install transformers datasets scikit-learn pandas numpy
!pip install --upgrade transformers

# Standard libraries
import os
import random

# Data manipulation libraries
import numpy as np
import pandas as pd

# PyTorch related
import torch
from torch.optim.swa_utils import AveragedModel
from safetensors.torch import load_file

# Hugging Face transformers
from transformers import (
    RobertaConfig,
    RobertaForSequenceClassification,
    RobertaTokenizer,
    Trainer,
    TrainerCallback,
    TrainingArguments
)
from datasets import Dataset

# Google Colab
from google.colab import drive

# Set Constant Seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(42)

# CSV Display
from IPython.display import display

# Mount Drive
drive.mount('/content/drive/')
save_path = '/content/drive/MyDrive/nlu'
os.makedirs(save_path, exist_ok=True)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# **Generate Prediction from One Pair**

In [11]:
# Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"

# Set the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Model
save_path = "/content/drive/MyDrive/nlu/swa_final_model_last"
config = RobertaConfig.from_pretrained(save_path)
model = RobertaForSequenceClassification(config)
state_dict = load_file(os.path.join(save_path, "model.safetensors"))
model.load_state_dict(state_dict)
model.to(device)
model.eval()
eval_trainer = Trainer(model=model)
print("Model loaded from safetensors successfully")

def tokenize(batch):
    return tokenizer(batch["premise"], batch["hypothesis"],
                     truncation=True, padding="max_length", max_length=128)

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(save_path)

# Define a custom sentence pair
premise = "A boy in a red shirt is riding his bicycle down the street."
hypothesis = "The boy is playing outside."

# Tokenize the custom sentence pair and move inputs to the same device
inputs = tokenizer(premise, hypothesis,
                   truncation=True, padding="max_length", max_length=128,
                   return_tensors="pt")
inputs = {key: val.to(device) for key, val in inputs.items()}

# Run inference
with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits

# Get prediction
prediction = torch.argmax(logits, dim=1).item()
print("Prediction:", prediction)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Model loaded from safetensors successfully
Prediction: 1


# **Generate Predictions from Dataset**

In [12]:
# Load the dataset from csv
dataset_df = pd.read_csv('/content/drive/MyDrive/nlu/dev.csv')
dataset_df["label"] = dataset_df["label"].astype(int)

dataset_ds = Dataset.from_pandas(dataset_df).map(tokenize, batched=True).rename_column("label", "labels")
dataset_ds.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

# Run inference on validation/dev dataset
pred_output = eval_trainer.predict(dataset_ds)
pred_logits = pred_output.predictions
pred_labels = np.argmax(pred_logits, axis=1)

# Build a DataFrame and save
pd.DataFrame({"prediction": pred_labels}).to_csv("predictions_demo.csv", index=False)
print("Saved predictions_demo.csv with", len(pred_labels), "rows.")

# Set options to display all rows and columns
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Load and display the predictions CSV
predictions_df = pd.read_csv("predictions_demo.csv")

display(predictions_df)

Map:   0%|          | 0/6736 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Saved predictions_demo.csv with 6736 rows.


,prediction
0,0
1,1
2,1
3,0
4,1
5,1
6,1
7,1
8,0
9,0
